In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import seaborn as sns
import pickle
#from src.score import * # load_test_data defined here
from collections import OrderedDict

from tensorflow.keras.layers import Input, UpSampling3D, AveragePooling3D, concatenate, ReLU, Reshape, Concatenate, \
    Permute
#from DLWP.custom import CubeSpherePadding2D, CubeSphereConv2D
from tensorflow.keras.layers import Reshape, Concatenate, Permute
from tensorflow.keras.models import Model
import glob
from tensorflow.keras.models import load_model

import os
import numpy as np

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
import cv2
from patchify import * 
#import tensorly as tl

In [2]:
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))
def mse(y, t):
    return np.mean(np.square(y - t))

def psnr(y, t, max_=200.0):
    return 20 * np.log10(max_) - 10 * np.log10(mse(y, t))

def ssim(x, y, max_=200.0):
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x)
    var_y = np.var(y)
    cov = np.mean((x - mu_x) * (y - mu_y))
    c1 = np.square(0.01 * max_)
    c2 = np.square(0.03 * max_)
    return ((2 * mu_x * mu_y + c1) * (2 * cov + c2)) / ((mu_x**2 + mu_y**2 + c1) * (var_x + var_y + c2))


# GPUs Available:  0


In [3]:
data_dir='../data/input_satellite_jaxa_gsmap'
file_ = glob.glob(data_dir+'GSMaP.v7_0.10deg_????-*_austin.nc')

In [4]:
ds = xr.open_mfdataset(file_, combine='by_coords')

In [5]:
ds_train = ds.sel(time=slice('2000', '2009'))
ds_test  = ds.sel(time=slice('2010', '2020'))
min_ = 0.0 # min_max.values[0]
max_ = 200.0 #min_max.values[1]

In [6]:
model = Sequential()
model.add(Conv2D(64,9,padding='same',input_shape=(20,20,1)))
model.add(Activation('relu'))
model.add(Conv2D(32,1,padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(1,5,padding='same'))
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
weight_filename = 'srcnn_weight_200321.hdf5'
model.load_weights(os.path.join('./model/',weight_filename))

In [8]:
for pass_ in range(7,1000):
    for year in range(2001,2009):
        # Load data
        print('Loading data ', year)
        data_ = ds_train.pr.sel(time=slice(str(year), str(year))).values

        print('Normalize')
        # Normalize
        data_n = ((data_[2:,:,:]-min_)/(max_-min_))

        print('Gen input and output]')
        # Generate input and output
        X = []
        y = []
        for i_ in range(data_n.shape[0]):
            img = data_n[i_,:,:]
            if np.sum(img)>0:
                scale = 2
                res = cv2.resize(img, dsize=(img.shape[0]*scale, img.shape[1]*scale), interpolation=cv2.INTER_CUBIC)
                res_ = cv2.resize(res, dsize=(int(res.shape[0]*(1/scale)), \
                                              int(res.shape[1]*(1/scale))), interpolation=cv2.INTER_CUBIC)
                #data_n_[i_,:,:] = res_
                X.append(img)
                y.append(res_)

        X = np.asarray(X)
        y = np.asarray(y) 

        print('Gen sub images')
        # Transform to sub images
        sub_images_ = []
        sub_labels_ = []
        for i_ in range(X.shape[0]):
            image = X[i_]
            patches = patchify(image, (20,20), step=10)
            sub_images_.append(patches)

            image = y[i_]
            patches = patchify(image, (20,20), step=10)
            sub_labels_.append(patches)

        sub_images = np.asarray(sub_images_).reshape(-1,20,20)
        sub_labels = np.asarray(sub_labels_).reshape(-1,20,20)

        X_train = sub_images[:,:,:,np.newaxis]
        y_train = sub_labels[:,:,:,np.newaxis]

        print('Training')
        history = model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_split=0.1)#, \
#                         callbacks=[tf.keras.callbacks.EarlyStopping(
#                             monitor='val_loss',
#                             min_delta=0,
#                             patience=2,
#                             verbose=1, 
#                             mode='auto'
#                         )])
        model.save_weights(os.path.join('./model/','srcnn_weight_'+str(year)+str(pass_)+'.hdf5'))
        np.save('history'+str(year)+'_.npy',history.history)
    
    # To load
    # history=np.load('my_history.npy',allow_pickle='TRUE').item()
    # https://stackoverflow.com/questions/41061457/keras-how-to-save-the-training-history-attribute-of-the-history-object


Loading data  2001
Normalize
Gen input and output]
Gen sub images
Training
Train on 14284 samples, validate on 1588 samples
Epoch 1/100
14284/14284 [==============================] - 38s 3ms/sample - loss: 5.1138e-08 - accuracy: 0.7193 - val_loss: 4.0299e-10 - val_accuracy: 0.6674
Epoch 2/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 2.8997e-10 - accuracy: 0.7193 - val_loss: 4.0918e-10 - val_accuracy: 0.6674
Epoch 3/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 2.7813e-10 - accuracy: 0.7193 - val_loss: 3.6277e-10 - val_accuracy: 0.6674
Epoch 4/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 3.2782e-10 - accuracy: 0.7193 - val_loss: 3.9439e-10 - val_accuracy: 0.6674
Epoch 5/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 9.6307e-09 - accuracy: 0.7193 - val_loss: 2.8462e-07 - val_accuracy: 0.6674
Epoch 6/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 2.50

Epoch 52/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 5.8338e-09 - accuracy: 0.7193 - val_loss: 6.2102e-10 - val_accuracy: 0.6674
Epoch 53/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 1.7195e-08 - accuracy: 0.7193 - val_loss: 5.8269e-09 - val_accuracy: 0.6674
Epoch 54/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 1.0580e-09 - accuracy: 0.7193 - val_loss: 5.3867e-10 - val_accuracy: 0.6674
Epoch 55/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 1.6781e-08 - accuracy: 0.7193 - val_loss: 2.0079e-09 - val_accuracy: 0.6674
Epoch 56/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 4.8311e-10 - accuracy: 0.7193 - val_loss: 3.5594e-10 - val_accuracy: 0.6674
Epoch 57/100
14284/14284 [==============================] - 37s 3ms/sample - loss: 5.0670e-10 - accuracy: 0.7193 - val_loss: 1.0138e-09 - val_accuracy: 0.6674
Epoch 58/100
14284/14284 [====================

Epoch 3/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 8.2366e-10 - accuracy: 0.7225 - val_loss: 8.4407e-10 - val_accuracy: 0.6999
Epoch 4/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 5.1731e-10 - accuracy: 0.7225 - val_loss: 8.0091e-10 - val_accuracy: 0.6999
Epoch 5/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 5.6441e-10 - accuracy: 0.7225 - val_loss: 5.8449e-10 - val_accuracy: 0.6999
Epoch 6/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 4.7218e-10 - accuracy: 0.7225 - val_loss: 6.4376e-10 - val_accuracy: 0.6999
Epoch 7/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 4.7405e-09 - accuracy: 0.7225 - val_loss: 1.2783e-08 - val_accuracy: 0.6999
Epoch 8/100
14457/14457 [==============================] - 38s 3ms/sample - loss: 1.0715e-08 - accuracy: 0.7225 - val_loss: 6.2260e-09 - val_accuracy: 0.6999
Epoch 9/100
14457/14457 [===========================

Epoch 5/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 5.0215e-09 - accuracy: 0.7131 - val_loss: 1.6695e-10 - val_accuracy: 0.8666
Epoch 6/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 1.5739e-08 - accuracy: 0.7131 - val_loss: 1.7194e-10 - val_accuracy: 0.8666
Epoch 7/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 7.6507e-10 - accuracy: 0.7131 - val_loss: 9.4820e-10 - val_accuracy: 0.8666
Epoch 8/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 6.5077e-10 - accuracy: 0.7131 - val_loss: 1.3649e-10 - val_accuracy: 0.8666
Epoch 9/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 4.6771e-09 - accuracy: 0.7131 - val_loss: 1.1601e-10 - val_accuracy: 0.8666
Epoch 10/100
15159/15159 [==============================] - 40s 3ms/sample - loss: 3.2987e-09 - accuracy: 0.7131 - val_loss: 1.1829e-08 - val_accuracy: 0.8666
Epoch 11/100
15159/15159 [=========================

Epoch 7/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 1.1278e-09 - accuracy: 0.6843 - val_loss: 4.8265e-09 - val_accuracy: 0.7920
Epoch 8/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 3.1479e-08 - accuracy: 0.6843 - val_loss: 1.9466e-09 - val_accuracy: 0.7920
Epoch 9/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 1.0763e-09 - accuracy: 0.6843 - val_loss: 8.4545e-10 - val_accuracy: 0.7920
Epoch 10/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 1.4775e-08 - accuracy: 0.6843 - val_loss: 1.7447e-09 - val_accuracy: 0.7920
Epoch 11/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 7.3666e-10 - accuracy: 0.6843 - val_loss: 8.6444e-10 - val_accuracy: 0.7920
Epoch 12/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 1.0934e-08 - accuracy: 0.6843 - val_loss: 1.0139e-09 - val_accuracy: 0.7920
Epoch 13/100
15912/15912 [=======================

...

Epoch 27/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 4.4412e-09 - accuracy: 0.6843 - val_loss: 2.9596e-09 - val_accuracy: 0.7920
Epoch 28/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 3.3497e-09 - accuracy: 0.6843 - val_loss: 1.2339e-08 - val_accuracy: 0.7920
Epoch 29/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 4.3371e-09 - accuracy: 0.6843 - val_loss: 1.3624e-09 - val_accuracy: 0.7920
Epoch 30/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 2.9756e-09 - accuracy: 0.6843 - val_loss: 1.2832e-08 - val_accuracy: 0.7920
Epoch 31/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 7.5996e-09 - accuracy: 0.6843 - val_loss: 5.7163e-10 - val_accuracy: 0.7920
Epoch 32/100
15912/15912 [==============================] - 41s 3ms/sample - loss: 7.5711e-09 - accuracy: 0.6843 - val_loss: 6.4615e-10 - val_accuracy: 0.7920
Epoch 33/100
15912/15912 [====================

Epoch 29/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 7.5039e-10 - accuracy: 0.7194 - val_loss: 7.7515e-10 - val_accuracy: 0.8174
Epoch 30/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 1.2877e-09 - accuracy: 0.7194 - val_loss: 1.2175e-09 - val_accuracy: 0.8174
Epoch 31/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 3.5106e-08 - accuracy: 0.7194 - val_loss: 1.8442e-09 - val_accuracy: 0.8174
Epoch 32/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 7.3647e-10 - accuracy: 0.7194 - val_loss: 2.8712e-10 - val_accuracy: 0.8174
Epoch 33/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 7.7535e-10 - accuracy: 0.7194 - val_loss: 2.9003e-10 - val_accuracy: 0.8174
Epoch 34/100
13586/13586 [==============================] - 35s 3ms/sample - loss: 6.3068e-10 - accuracy: 0.7194 - val_loss: 3.3520e-10 - val_accuracy: 0.8174
Epoch 35/100
13586/13586 [====================

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/lus/dal/cccr_rnd/manmeet/anaconda3/envs/causal/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-c8a123051c34>", line 49, in <module>
    history = model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_split=0.1)#, \
  File "/lus/dal/cccr_rnd/manmeet/anaconda3/envs/causal/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 728, in fit
    use_multiprocessing=use_multiprocessing)
  File "/lus/dal/cccr_rnd/manmeet/anaconda3/envs/causal/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 324, in fit
    total_epochs=epochs)
  File "/lus/dal/cccr_rnd/manmeet/anaconda3/envs/causal/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 123, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/lus/dal

TypeError: object of type 'NoneType' has no len()